# Dependencies

In [ ]:
!pip install numpy==1.22.4
!pip install matplotlib==3.7.1
!pip install mglearn==0.2.0
!pip install pandas==2.0.3
!pip install scikit-learn==1.3.2
!pip install seaborn==0.13.1

In [ ]:
!pip freeze

# Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
from IPython.display import display
import seaborn as sns
from sklearn import datasets
from sklearn.cluster import KMeans

In [2]:
df=pd.read_csv("airline_satisfaction.csv")

In [3]:
y = df.satisfaction.values

categorical_variables = ['Gender','Customer Type','Type of Travel','Class']

quantitative_variables = ['Age','Flight Distance', 'Departure Delay in Minutes','Arrival Delay in Minutes']

ordinal_variables = ['Inflight wifi service', 'Departure/Arrival time convenient','Ease of Online booking', 
                     'Gate location','Food and drink','Online boarding','Seat comfort','Inflight entertainment',
                     'On-board service','Leg room service','Baggage handling','Checkin service','Inflight service',
                     'Cleanliness']

X = df[categorical_variables + quantitative_variables + ordinal_variables].values
X_quantitative = df[quantitative_variables].values
X_categorical = df[categorical_variables].values
X_ordinal = df[ordinal_variables].values

In [4]:
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


# Data Preprocessing

## Encoding Variables

### One-Hot Encoding
for categorical variables

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
one_hot_encoder = OneHotEncoder()

In [7]:
one_hot_encoder.fit(X_categorical)
one_hot_encoded_X = one_hot_encoder.transform(X_categorical)

In [8]:
one_hot_encoded_X.toarray()

array([[0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 1., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.]])

In [9]:
one_hot_encoder.get_feature_names_out()

array(['x0_Female', 'x0_Male', 'x1_Loyal Customer',
       'x1_disloyal Customer', 'x2_Business travel', 'x2_Personal Travel',
       'x3_Business', 'x3_Eco', 'x3_Eco Plus'], dtype=object)

In [10]:
one_hot_encoded_X

<129880x9 sparse matrix of type '<class 'numpy.float64'>'
	with 519520 stored elements in Compressed Sparse Row format>

In [11]:
df_categorical=df[categorical_variables]

In [12]:
categorical_cols = df.select_dtypes(include=['object']).columns
df_encoded = pd.get_dummies(df, columns=categorical_cols)

print("Number of new columns generated after one-hot encoding:", df_encoded.shape[1] - df.shape[1])

Number of new columns generated after one-hot encoding: 6


In [13]:
X_categorical_encoded = pd.concat([df_categorical, df_encoded], axis=1)

In [14]:
X_categorical_encoded

,Gender,Customer Type,Type of Travel,Class,Unnamed: 0,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_neutral or dissatisfied,satisfaction_satisfied
0,Male,Loyal Customer,Personal Travel,Eco Plus,0,70172,13,460,3,4,...,True,True,False,False,True,False,False,True,True,False
1,Male,disloyal Customer,Business travel,Business,1,5047,25,235,3,2,...,True,False,True,True,False,True,False,False,True,False
2,Female,Loyal Customer,Business travel,Business,2,110028,26,1142,2,2,...,False,True,False,True,False,True,False,False,False,True
3,Female,Loyal Customer,Business travel,Business,3,24026,25,562,2,5,...,False,True,False,True,False,True,False,False,True,False
4,Male,Loyal Customer,Business travel,Business,4,119299,61,214,3,3,...,True,True,False,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129875,Male,disloyal Customer,Business travel,Business,25971,78463,34,526,3,3,...,True,False,True,True,False,True,False,False,True,False
129876,Male,Loyal Customer,Business travel,Business,25972,71167,23,646,4,4,...,True,True,False,True,False,True,False,False,False,True
129877,Female,Loyal Customer,Personal Travel,Eco,25973,37675,17,828,2,5,...,False,True,False,False,True,False,True,False,True,False
129878,Male,Loyal Customer,Business travel,Business,25974,90086,14,1127,3,3,...,True,True,False,True,False,True,False,False,False,True


### Ordinal Encoding

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
label_encoder = LabelEncoder()

In [17]:
X_ordinal_encoded = np.zeros_like(X_ordinal)

In [18]:
for i in range(X_ordinal.shape[1]):
    X_ordinal_encoded[:, i] = label_encoder.fit_transform(X_ordinal[:, i])

In [19]:
for i, col in enumerate(ordinal_variables):
    print(f"===== Encoded classes for {col}: {label_encoder.classes_}")

===== Encoded classes for Inflight wifi service: [0 1 2 3 4 5]
===== Encoded classes for Departure/Arrival time convenient: [0 1 2 3 4 5]
===== Encoded classes for Ease of Online booking: [0 1 2 3 4 5]
===== Encoded classes for Gate location: [0 1 2 3 4 5]
===== Encoded classes for Food and drink: [0 1 2 3 4 5]
===== Encoded classes for Online boarding: [0 1 2 3 4 5]
===== Encoded classes for Seat comfort: [0 1 2 3 4 5]
===== Encoded classes for Inflight entertainment: [0 1 2 3 4 5]
===== Encoded classes for On-board service: [0 1 2 3 4 5]
===== Encoded classes for Leg room service: [0 1 2 3 4 5]
===== Encoded classes for Baggage handling: [0 1 2 3 4 5]
===== Encoded classes for Checkin service: [0 1 2 3 4 5]
===== Encoded classes for Inflight service: [0 1 2 3 4 5]
===== Encoded classes for Cleanliness: [0 1 2 3 4 5]


In [20]:
X_ordinal_encoded

array([[3, 4, 3, ..., 4, 5, 5],
       [3, 2, 3, ..., 1, 4, 1],
       [2, 2, 2, ..., 4, 4, 5],
       ...,
       [2, 5, 1, ..., 5, 4, 2],
       [3, 3, 3, ..., 4, 5, 4],
       [2, 5, 2, ..., 1, 1, 1]])

In [21]:
X_ordinal_encoded.shape

(129880, 14)

In [22]:
X_ordinal.shape

(129880, 14)

## Creating new Dataset

In [23]:
df_quantitative = pd.DataFrame(X_quantitative, columns=['Age','Flight Distance', 'Departure Delay in Minutes','Arrival Delay in Minutes'])

In [24]:
df_quantitative

,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes
0,13.0,460.0,25.0,18.0
1,25.0,235.0,1.0,6.0
2,26.0,1142.0,0.0,0.0
3,25.0,562.0,11.0,9.0
4,61.0,214.0,0.0,0.0
...,...,...,...,...
129875,34.0,526.0,0.0,0.0
129876,23.0,646.0,0.0,0.0
129877,17.0,828.0,0.0,0.0
129878,14.0,1127.0,0.0,0.0


In [25]:
df_categorical_encoded = pd.DataFrame(X_categorical_encoded, columns=['Gender_Male','Customer Type_Loyal Customer', 'Customer Type_disloyal Customer', 'Type of Travel_Business travel', 'Type of Travel_Personal Travel', 'Class_Business', 'Class_Eco', 'Class_Eco Plus', 'satisfaction_neutral or dissatisfied', 'satisfaction_satisfied'])

In [26]:
df_categorical_encoded

,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_neutral or dissatisfied,satisfaction_satisfied
0,True,True,False,False,True,False,False,True,True,False
1,True,False,True,True,False,True,False,False,True,False
2,False,True,False,True,False,True,False,False,False,True
3,False,True,False,True,False,True,False,False,True,False
4,True,True,False,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...
129875,True,False,True,True,False,True,False,False,True,False
129876,True,True,False,True,False,True,False,False,False,True
129877,False,True,False,False,True,False,True,False,True,False
129878,True,True,False,True,False,True,False,False,False,True


In [27]:
df_ordinal_encoded = pd.DataFrame(X_ordinal_encoded, columns=['Inflight wifi service','Departure/Arrival time convenient','Ease of Online booking','Gate location','Food and drink','Online boarding','Seat comfort','Inflight entertainment','On-board service','Leg room service','Baggage handling','Checkin service','Inflight service','Cleanliness'])

In [28]:
df_ordinal_encoded

,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness
0,3,4,3,1,5,3,5,5,4,3,3,4,5,5
1,3,2,3,3,1,3,1,1,1,5,2,1,4,1
2,2,2,2,2,5,5,5,5,4,3,3,4,4,5
3,2,5,5,5,2,2,2,2,2,5,2,1,4,2
4,3,3,3,3,4,5,5,3,3,4,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129875,3,3,3,1,4,3,4,4,3,2,3,4,5,4
129876,4,4,4,4,4,4,4,4,4,5,4,5,5,4
129877,2,5,1,5,2,1,2,2,4,3,3,5,4,2
129878,3,3,3,3,4,4,4,4,3,2,4,4,5,4


In [33]:
df_combined = pd.concat([df_quantitative, df_categorical_encoded, df_ordinal_encoded], axis=1)

In [34]:
print(df_combined)

         Age  Flight Distance  Departure Delay in Minutes  \
0       13.0            460.0                        25.0   
1       25.0            235.0                         1.0   
2       26.0           1142.0                         0.0   
3       25.0            562.0                        11.0   
4       61.0            214.0                         0.0   
...      ...              ...                         ...   
129875  34.0            526.0                         0.0   
129876  23.0            646.0                         0.0   
129877  17.0            828.0                         0.0   
129878  14.0           1127.0                         0.0   
129879  42.0            264.0                         0.0   

        Arrival Delay in Minutes  Gender_Male  Customer Type_Loyal Customer  \
0                           18.0         True                          True   
1                            6.0         True                         False   
2                            0

In [41]:
df_combined.to_csv('/home/estelle/robotlearn/airline_satisfaction/airline_satisfaction_encoded.csv')

## Imputing Missing Values

In [35]:
from sklearn.impute import SimpleImputer

In [36]:
simple_imputer = SimpleImputer(strategy="mean")
simple_imputer.fit(X_quantitative)

X_imputed = simple_imputer.transform(X)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'Male'

In [ ]:
X_imputed

trash

In [ ]:
model = KMeans(n_clusters=3)

In [ ]:
model.fit(X_quantitative)

In [ ]:
all_predictions = model.predict(df.data)

In [ ]:
print(all_predictions)

In [ ]:
from sklearn import datasets
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
model = TSNE(learning_rate=100)

In [ ]:
transformed = model.fit_transform(df)

In [ ]:
x_axis = transformed[:, 0]
y_axis = transformed[:, 1]

plt.scatter(x_axis, y_axis, c=iris_df.target)
plt.show()